In [1]:
import pandas as pd
import requests

In [2]:
url="https://api.census.gov/data/2019/pep/charagegroups?"


In [5]:
all_population="000"
age_65_up="026"
query_url=url+ "get=NAME,POP&for=county:*&in=state:12&AGEGROUP="
all_pop_query=query_url+all_population
age_65_up_query=query_url+age_65_up
results_cols=["County, State","Population","Age Group","State Code","County Code"]

In [6]:
all_pop_results=requests.get(all_pop_query).json()
age_65_up_results=requests.get(age_65_up_query).json()

In [7]:
age_65_up_df=pd.DataFrame(age_65_up_results,columns=results_cols)
age_65_up_df.columns=age_65_up_df.iloc[0]
age_65_up_df=age_65_up_df.tail(-1)
age_65_up_df

,NAME,POP,AGEGROUP,state,county
1,"Washington County, Florida",4380,26,12,133
2,"Duval County, Florida",138594,26,12,031
3,"Bradford County, Florida",5064,26,12,007
4,"Brevard County, Florida",145752,26,12,009
5,"Clay County, Florida",35844,26,12,019
...,...,...,...,...,...
63,"Hernando County, Florida",53348,26,12,053
64,"Hillsborough County, Florida",214035,26,12,057
65,"Okaloosa County, Florida",34376,26,12,091
66,"Taylor County, Florida",4525,26,12,123


In [13]:
all_pop_df=pd.DataFrame(all_pop_results,columns=results_cols)
all_pop_df.columns=all_pop_df.iloc[0]
all_pop_df=all_pop_df.tail(-1)
all_pop_df

,NAME,POP,AGEGROUP,state,county
1,"Washington County, Florida",25473,0,12,133
2,"Duval County, Florida",957755,0,12,031
3,"Bradford County, Florida",28201,0,12,007
4,"Brevard County, Florida",601942,0,12,009
5,"Clay County, Florida",219252,0,12,019
...,...,...,...,...,...
63,"Hernando County, Florida",193920,0,12,053
64,"Hillsborough County, Florida",1471968,0,12,057
65,"Okaloosa County, Florida",210738,0,12,091
66,"Taylor County, Florida",21569,0,12,123


In [34]:
pop_percent_df=pd.merge(all_pop_df,age_65_up_df,on='county',how='outer')

In [35]:
#lambda to split name and remove the county.
pop_percent_df[['County','State']]=pop_percent_df['NAME_x'].str.split(', ',expand=True)
pop_percent_df.drop(columns=['NAME_x','NAME_y','AGEGROUP_x','AGEGROUP_y','state_x','state_y'],inplace=True)

In [43]:
pop_percent_df=pop_percent_df.rename(columns={'county':'County_ID','POP_x':'Total_Population','POP_y':'Elderly_Population','state':'State_ID'})
data_types={'Total_Population':'int','County_ID':'int','Elderly_Population':'int','County':str,'State':str}
pop_percent_df=pop_percent_df.astype(data_types)
pop_percent_df['Percent_Elderly']=(pop_percent_df['Elderly_Population']/pop_percent_df['Total_Population'])*100
pop_percent_df['Percent_Elderly'] = pop_percent_df['Percent_Elderly'].map("{:.2f}".format)
pop_percent_df['County'] = pop_percent_df['County'].str.replace(' County', '')
pop_percent_df

,Total_Population,County_ID,Elderly_Population,County,State,Percent_Elderly
0,25473,133,4380,Washington,Florida,17.19
1,957755,31,138594,Duval,Florida,14.47
2,28201,7,5064,Bradford,Florida,17.96
3,601942,9,145752,Brevard,Florida,24.21
4,219252,19,35844,Clay,Florida,16.35
...,...,...,...,...,...,...
62,193920,53,53348,Hernando,Florida,27.51
63,1471968,57,214035,Hillsborough,Florida,14.54
64,210738,91,34376,Okaloosa,Florida,16.31
65,21569,123,4525,Taylor,Florida,20.98


In [11]:
county_table=df[['County_ID','County']]
state_table=df[['State_ID','State']]
county_table

,County_ID,County
1,133,Washington County
2,031,Duval County
3,007,Bradford County
4,029,Dixie County
5,033,Escambia County
...,...,...
63,127,Volusia County
64,081,Manatee County
65,075,Levy County
66,073,Leon County
